<a href="https://colab.research.google.com/github/mohamedseklani/DLI/blob/main/1_data_cleaning_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git config --global user.email "mohamedseklani8@gmail.com"
!git config --global user.name "mohamedseklani"

!git clone https://github.com/mohamedseklani/DLI
%cd DLI

fatal: destination path 'DLI' already exists and is not an empty directory.
/content/DLI


In [4]:
# --- Commit 1: Initial load and null cleanup ---

# 1) Setup
import pandas as pd
import numpy as np
import csv, sys

# Increase limit for very long email bodies
csv.field_size_limit(sys.maxsize)

# Pandas display settings
pd.set_option('display.max_columns', None)

# 2) Data Loading (robust)
# Using on_bad_lines='skip' instead of deprecated error_bad_lines
df = pd.read_csv(
    "/content/DLI/TREC-05.csv",  # <-- adjust path to your file
    engine="python",
    on_bad_lines="skip",
    sep=None,  # autodetect delimiter
    encoding="utf-8",
    encoding_errors="replace",  # handle weird characters
    dtype=str  # read all as string first
)

print("Initial Shape:", df.shape)
print(df.isnull().sum())

# 3) Preprocessing - Remove nulls in key columns
# Convert label and urls to numeric (coerce errors to NaN)
if "label" in df.columns:
    df["label"] = pd.to_numeric(df["label"], errors="coerce")
if "urls" in df.columns:
    df["urls"] = pd.to_numeric(df["urls"], errors="coerce")

# Drop rows where body, subject, or label is missing
required_cols = [c for c in ["body", "subject", "label"] if c in df.columns]
df.dropna(subset=required_cols, inplace=True)

# Fill missing urls with 0 and ensure integer type
if "urls" in df.columns:
    df["urls"] = df["urls"].fillna(0)
    df["urls"] = np.maximum(df["urls"].astype(float), 0).astype(int)

# Strip extra whitespace from text fields
for col in ["subject", "body"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

df.reset_index(drop=True, inplace=True)
print("After cleaning Shape:", df.shape)

Initial Shape: (53875, 7)
sender        13
receiver    2627
date        2376
subject     2472
body        1214
label       1341
urls        1342
dtype: int64
After cleaning Shape: (51264, 7)


In [10]:
import os

os.makedirs("data", exist_ok=True)
os.makedirs("notebooks", exist_ok=True)

# Save cleaned CSV inside data/
df.to_csv("data/cleaned_data.csv", index=False)

In [13]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   data/cleaned_data.csv

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	TREC-05.csv

